In [9]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from IPython.display import display
from joblib import dump
from keras.utils.vis_utils import plot_model
from sklearn import set_config

from sklearn.metrics import (ConfusionMatrixDisplay, classification_report,
                             f1_score, make_scorer, confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier, plot_tree

from ml_model import PipelineManager, PredictionPipeline
from utils.GPU_models import KerasClassifierModel, multi_input_embedded_model
from utils.preprocessing_utils import (clean_and_preprocess_datasets,
                                       clean_labs, clean_notas,
                                       clean_sociodemograficos,
                                       disease_tests_list, merge_classes,
                                       merge_labs_notas,
                                       word_count_feat_engineering)


In [10]:
set_config(display="diagram")
sns.set_style("darkgrid")
as_dual_class=False
target_feature = 'Código'
text_feature = 'Plan'
retrain_with_class_weight=False
consolidate_classes = False
cv = 3
n_iter = 6
n_jobs = -3

# False, 'oversample', or 'undersample'
balance_classes = 'oversample'
save_path = Path('data') / 'output' / 'best_model.pickle'

## Data setup

In [11]:
df_notas = pd.read_csv('../data/notas.csv', sep=';')
df_laboratorios = pd.read_csv('../data/laboratorios.csv', sep=';')
df_sociodemografico = pd.read_csv('../data/sociodemografico.csv', sep=';')

In [12]:
df_notas.head()

IDRecord Código                                             Nombre  \
0    44600   A539                           SIFILIS, NO ESPECIFICADA   
1    45038   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
2    40391   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
3   106350   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
4   105840   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   

                  Tipo                                               Plan  
0  Confirmado Repetido  - ORDENO TAR ABC +3TC +ATV/r  - PROFILAXIS NO ...  
1  Confirmado Repetido  - TAF/FTC/EVG/C MIPRES POR 2 MESES 20200602158...  
2  Confirmado Repetido  usuaria la cual se ve pertinente seguimiento d...  
3  Confirmado Repetido  1. Se formula TAR (TDF/FTC+EFV)  2. S/S Paracl...  
4  Confirmado Repetido  EDUCACIÓN  Se brinda retroalimentación con rel...

In [13]:
df_laboratorios.head()

IDRecord  Codigo                      Nombre             Fecha Valor
0     95627  902045  TIEMPO DE PROTROMBINA (PT)  22/02/2022 18:43   NaN
1    125572  902045  TIEMPO DE PROTROMBINA (PT)  17/02/2022 13:41   NaN
2     55788  902045  TIEMPO DE PROTROMBINA (PT)  22/06/2021 12:50  1.05
3    113766  902045  TIEMPO DE PROTROMBINA (PT)   5/08/2021 12:11   NaN
4     44596  902045  TIEMPO DE PROTROMBINA (PT)   5/08/2021 13:15   NaN

In [14]:
df_sociodemografico.head()

IDRecord  Edad  Genero                GrupoEtnico AreaResidencial  \
0         5    39   Mujer                    Mestizo     Zona Urbana   
1       292    84  Hombre  Ninguno de los anteriores     Zona Urbana   
2       300    88  Hombre                    Mestizo     Zona Urbana   
3       307    88  Hombre  Ninguno de los anteriores     Zona Urbana   
4       325    94  Hombre  Ninguno de los anteriores      Zona Rural   

    EstadoCivil TSangre  
0      Separado     NaN  
1        Casado     NaN  
2       Soltero      O+  
3  No reportado     NaN  
4       Viudo/a     NaN

In [15]:
df_notas.head()

IDRecord Código                                             Nombre  \
0    44600   A539                           SIFILIS, NO ESPECIFICADA   
1    45038   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
2    40391   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
3   106350   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
4   105840   A530  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   

                  Tipo                                               Plan  
0  Confirmado Repetido  - ORDENO TAR ABC +3TC +ATV/r  - PROFILAXIS NO ...  
1  Confirmado Repetido  - TAF/FTC/EVG/C MIPRES POR 2 MESES 20200602158...  
2  Confirmado Repetido  usuaria la cual se ve pertinente seguimiento d...  
3  Confirmado Repetido  1. Se formula TAR (TDF/FTC+EFV)  2. S/S Paracl...  
4  Confirmado Repetido  EDUCACIÓN  Se brinda retroalimentación con rel...

## Data Cleaning

In [16]:
df_sociodemografico = clean_sociodemograficos(df_sociodemografico)
df_laboratorios = clean_labs(df_laboratorios)
df_notas = clean_notas(df_notas, apply_lemmatization=False)

### Merge the sociodemographic data with the medical notes dataset

In [17]:
df_merge = df_sociodemografico.merge(df_notas, how='inner', on='IDRecord')
df_merge

IDRecord  Edad  Genero                GrupoEtnico AreaResidencial  \
0              5    39   Mujer                    Mestizo     Zona Urbana   
1            292    84  Hombre  Ninguno de los anteriores     Zona Urbana   
2            300    88  Hombre                    Mestizo     Zona Urbana   
3            300    88  Hombre                    Mestizo     Zona Urbana   
4            300    88  Hombre                    Mestizo     Zona Urbana   
...          ...   ...     ...                        ...             ...   
140167    205218    28  Hombre  Ninguno de los anteriores     Zona Urbana   
140168    205227    24  Hombre  Ninguno de los anteriores     Zona Urbana   
140169    205253    84  Hombre                    Mestizo     Zona Urbana   
140170    205577    62  Hombre                    Mestizo     Zona Urbana   
140171    206307    57  Hombre                    Mestizo     Zona Urbana   

        EstadoCivil TSangre Código  \
0          Separado     NaN   E109   
1            Casado     NaN   E119   
2           Soltero      O+   E119   
3           Soltero      O+   E109   
4           Soltero      O+   E119   
...             ...     ...    ...   
140167          NaN     NaN   A539   
140168      Soltero      O+   A530   
140169       Casado     NaN   E109   
140170  Desconocido     NaN   E119   
140171  Desconocido     NaN   E149   

                                                   Nombre  \
0       DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCI...   
1       DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...   
2       DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...   
3       DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCI...   
4       DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...   
...                                                   ...   
140167                           SIFILIS, NO ESPECIFICADA   
140168  SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...   
140169  DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCI...   
140170  DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...   
140171  DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION...   

                         Tipo  \
0         Confirmado Repetido   
1         Confirmado Repetido   
2         Confirmado Repetido   
3         Confirmado Repetido   
4            Confirmado Nuevo   
...                       ...   
140167    Confirmado Repetido   
140168    Confirmado Repetido   
140169    Confirmado Repetido   
140170  Impresión Diagnóstica   
140171    Confirmado Repetido   

                                                     Plan  
0       PACIENTE QUIEN CONTINUA EN PROGRAMA DE NEFROPR...  
1                      CONTINUA EN PROGRAMA DE CRONICOS.   
2       1- CONTINUAR EN PAD 2 - RECOMENDACIONES DE DIE...  
3       1- CONTINUAR EN PAD 2 - RECOMENDACIONES DE DIE...  
4       1- CONTINUAR EN PAD 2 - RECOMENDACIONES DE DIE...  
...                                                   ...  
140167  se explica acerca del programa, Se recomienda ...  
140168  Elaboracion duelo frente al diagnostico.   Ref...  
140169  FUROATO MOMETASONA 1 SPRY NASAL POR CADA FOSA ...  
140170  CONTROL MEICO EN UN MES-INFECTOLOGIA  VALORACI...  
140171  CONTINUA EN SEGUIMIENTO POR PROGRAMA NEFROPROT...  

[140172 rows x 11 columns]

## Feature Engineering

In [18]:
# Merge the classes
if consolidate_classes:
    df_merge = merge_classes(df_merge)

# Perform word count feature engineering
df_merge = word_count_feat_engineering(df_merge)

# Preprocess the lab data and merge it with the sociodemographic data
df_merge = merge_labs_notas(df_laboratorios, df_merge)

with pd.option_context('display.max_columns', None):
    display(df_merge.head(5))

IDRecord  Edad  Genero                GrupoEtnico AreaResidencial  \
0         5    39   Mujer                    Mestizo     Zona Urbana   
1       292    84  Hombre  Ninguno de los anteriores     Zona Urbana   
2       300    88  Hombre                    Mestizo     Zona Urbana   
3       300    88  Hombre                    Mestizo     Zona Urbana   
4       300    88  Hombre                    Mestizo     Zona Urbana   

  EstadoCivil TSangre Código  \
0    Separado     NaN   E109   
1      Casado     NaN   E119   
2     Soltero      O+   E119   
3     Soltero      O+   E109   
4     Soltero      O+   E119   

                                              Nombre                 Tipo  \
0  DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCI...  Confirmado Repetido   
1  DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...  Confirmado Repetido   
2  DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...  Confirmado Repetido   
3  DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCI...  Confirmado Repetido   
4  DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...     Confirmado Nuevo   

                                                Plan  acido  antibio  \
0  PACIENTE QUIEN CONTINUA EN PROGRAMA DE NEFROPR...      0        0   
1                 CONTINUA EN PROGRAMA DE CRONICOS.       0        0   
2  1- CONTINUAR EN PAD 2 - RECOMENDACIONES DE DIE...      0        0   
3  1- CONTINUAR EN PAD 2 - RECOMENDACIONES DE DIE...      0        0   
4  1- CONTINUAR EN PAD 2 - RECOMENDACIONES DE DIE...      0        0   

   asintoma  cabeza  diabet  diet  gluco  hepat  insulin  keto  penici  \
0         0       0       0     0      0      0        0     0       0   
1         0       0       0     0      0      0        0     0       0   
2         0       0       0     1      0      0        0     0       0   
3         0       0       0     1      0      0        0     0       0   
4         0       0       0     1      0      0        0     0       0   

   preservativo  rpr  sable  serolo  sifili  test_reloj_orden  vih  \
0             0    0      0       0       0                 0    0   
1             0    0      0       0       0                 0    0   
2             0    0      0       0       0                 0    0   
3             0    0      0       0       0                 0    0   
4             0    0      0       0       0                 0    0   

  top_lab_code  top_lab_avg_value  top_lab_max_value  top_lab_count  \
0       902213               10.0               10.0            1.0   
1          NaN                NaN                NaN            NaN   
2          NaN                NaN                NaN            NaN   
3          NaN                NaN                NaN            NaN   
4          NaN                NaN                NaN            NaN   

   total_lab_count  first_lab_date  last_lab_date  date_diff_first_last  \
0              8.0    1.609114e+18   1.609114e+18                   0.0   
1              NaN             NaN            NaN                   NaN   
2              NaN             NaN            NaN                   NaN   
3              NaN             NaN            NaN                   NaN   
4              NaN             NaN            NaN                   NaN   

   date_diff_mean  date_diff_max  liver_damage_count  liver_damage_max  \
0             0.0            0.0                 0.0               0.0   
1             NaN            NaN                 NaN               NaN   
2             NaN            NaN                 NaN               NaN   
3             NaN            NaN                 NaN               NaN   
4             NaN            NaN                 NaN               NaN   

   hemoglobin_count  hemoglobin_max  lymphocytes_count  lymphocytes_max  \
0               1.0            10.0                0.0              0.0   
1               NaN             NaN                NaN              NaN   
2               NaN             NaN                NaN             

## Label encoding

In [19]:
X = df_merge.drop(labels=[target_feature], axis=1)
y = df_merge[target_feature]
if as_dual_class:
    y = y.str[:2]

label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(y)

np.unique(y_labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([  977,    94,  2614,  1970, 60586, 47408,  6278, 17437,  2808]))

## Train-test split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, train_size=0.2, random_state=42, stratify=y_labels)
np.unique(y_train, return_counts=True) # Let's check the number of samples per label

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([  195,    19,   523,   394, 12117,  9481,  1256,  3487,   562]))

## Class Balancing

In [21]:
if balance_classes == 'oversample':
    # Using a naive oversampling approach
    sampler = RandomOverSampler(random_state=42)
    X_train, y_train = sampler.fit_resample(X_train, y_train)
elif balance_classes == 'undersample':
    # Using a naive oversampling approach
    sampler = RandomUnderSampler(random_state=42)
    X_train, y_train = sampler.fit_resample(X_train, y_train)
np.unique(y_train, return_counts=True) # Let's check the number of samples per label

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([12117, 12117, 12117, 12117, 12117, 12117, 12117, 12117, 12117]))

## Hyperparameter Setup

In [22]:
# Select the numerical features that will be used in the model
numerical_features = list(
    set(
        [
            "Edad",
            "top_lab_avg_value",
            "top_lab_max_value",
            "top_lab_count",
            "total_lab_count",
            "date_diff_mean",
            "date_diff_max",
            "first_lab_date",	
            "last_lab_date",
            "date_diff_first_last",
        ]
        + list(df_merge.drop(columns="IDRecord").select_dtypes(include="int64").columns)
        + [f'{test[1]}_count' for test in disease_tests_list()]
        + [f'{test[1]}_max' for test in disease_tests_list()]
    )
)

# Now select the categorical features
categorical_features = [
    "Genero",
    "GrupoEtnico",
    # "AreaResidencial",
    "EstadoCivil",
    # "TSangre",
    # "Tipo",
    "top_lab_code",
]

In [23]:
pipeline = PipelineManager(estimator="classifier")
pipeline.set_numerical_features(numerical_features)
pipeline.set_categorical_features(categorical_features)
pipeline.set_text_feature(text_feature)
pipeline.set_basic_pipeline()

param_grid = {
    "n_estimators": np.linspace(1, 100, 10, dtype=int),
    "max_depth": list(np.linspace(2, 15, 5, dtype=int)),
    "eta": np.linspace(0.01, 0.5, 10, dtype=float),
    "min_child_weight": np.linspace(0.5, 20, 5, dtype=float),
    "gamma": np.linspace(0, 1, 5, dtype=float),
    "subsample": np.linspace(0.1, 1, 5, dtype=float),
    "colsample_bytree": np.linspace(0.2, 1, 5, dtype=float),
    "reg_lambda": np.linspace(0, 10, 5, dtype=float),
    "reg_alpha": np.linspace(0, 10, 5, dtype=float),
    # "scale_pos_weight": np.linspace(0.1, 500, 100, dtype=float),
}
estimator = XGBClassifier()
pipeline.add_estimator(estimator, param_grid)

param_grid = {
    "C": np.linspace(0, 5, 20, dtype=float),
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["auto", "scale"],
    "class_weight": ["balanced", None],
    "coef0": np.linspace(0, 5, 20, dtype=float),
    "degree": np.linspace(1, 5, 10, dtype=int),
}

pipeline.pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Genero', 'GrupoEtnico',
                                                   'EstadoCivil',
                                                   'top_lab_code']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScal...
                                                                   CountVectorizer(stop_words=['de',
                                                                                               'la',
                                                                                               'que',
                                                                                               'el',
                                                                                               'en',
                                                                                               'y',
                                                                                               'a',
                                                                                               'los',
                                                                                               'del',
                                                                                               'se',
                                                                                               'las',
                                                                                               'por',
                                                                                               'un',
                                                                                               'para',
                                                                                               'con',
                                                                                               'no',
                                                                                               'una',
                                                                                               'su',
                                                                                               'al',
                                                                                               'lo',
                                                                                               'como',
                                                                                               'mas',
                                                                                               'pero',
                                                                                               'sus',
                                                                                               'le',
                                                                                               'ya',
                                                                                               'o',
                                                                                             

## Hyperparameter tuning

In [24]:
sample_weight = compute_sample_weight(
    class_weight='balanced',
    y=df_merge[target_feature])

scoring = {
    "Accuracy": "balanced_accuracy",
    "Weighted_F1": make_scorer(f1_score, average='weighted'),
    # 'roc_auc':make_scorer(roc_auc_score, average='weighted'),
    }
best_model = pipeline.find_best_model(
    X_train,
    y_train,
    cv=cv,
    n_iter=n_iter,
    n_jobs=n_jobs,
    scoring=scoring,
    random_state=7,
    refit='Weighted_F1',
    verbose = 8,
    # error_score='raise',
    )

preprocessor = best_model.named_steps['preprocessor']
preprocessor

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 18.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.8/dist-packages/xgboost/core.py", line 532, in inner_f
    return f(**kwargs)
  File "/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.p

In [31]:
with pd.option_context('display.max_columns', None):
    display(pipeline.cv_results.sort_values(by=["rank_test_Weighted_F1"]).head(30))

AttributeError: 'PipelineManager' object has no attribute 'cv_results'

In [26]:
X_train_preprocessed = pd.DataFrame.sparse.from_spmatrix(preprocessor.transform(X_train), columns=list(preprocessor.get_feature_names_out()))
X_test_preprocessed = pd.DataFrame.sparse.from_spmatrix(preprocessor.transform(X_test), columns=list(preprocessor.get_feature_names_out()))

NameError: name 'preprocessor' is not defined

In [ ]:
# embedding = "nnlm-es-dim128"
embedding = "nnlm-es-dim128-with-normalization"
# embedding = "https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_L-24_H-1024_A-16/1"


model_function = multi_input_embedded_model
clf = KerasClassifierModel(
    build_fn=model_function,
    class_number=len(np.unique(y_train)),
    feature_number=len(X_train_preprocessed.columns),
    embedding = embedding,
    activation='relu',
    epochs=1000,
    batch_size=200,
    verbose=10,
)
model = model_function(
    class_number=len(np.unique(y_train)),
    feature_number=len(X_train_preprocessed.columns),
    embedding = embedding,
    ).summary()

In [ ]:
hist = clf.fit([X_train_preprocessed, X_train[text_feature]], y_train)
clf.plot_learning_curves('data/output/gpu_model')

In [ ]:
model_plot_savepath = Path('data') / 'output' / 'gpu_model' / 'model_plot.png'
model_plot_savepath.parent.mkdir(parents=True, exist_ok=True)
plot_model(clf.model, to_file=str(model_plot_savepath), show_shapes=True, show_layer_names=True)
# clf.model

## Scoring

In [ ]:
labels_decoded = label_encoder.inverse_transform(np.unique(y_train))
y_pred = clf.predict([X_test_preprocessed, X_test[text_feature]])

In [ ]:
class_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, labels=np.unique(y_train), target_names=labels_decoded, zero_division=False)).transpose().round(2)
class_report

In [ ]:
test_confusion_matrix = confusion_matrix(y_test, y_pred)

with sns.axes_style("dark"):
    fig, ax = plt.subplots(figsize=(20,10))
    disp = ConfusionMatrixDisplay(confusion_matrix=test_confusion_matrix,
                                display_labels=labels_decoded)
    disp.plot(ax=ax, cmap='viridis')
    plt.show()

In [ ]:
score = {'confusion_matrix':test_confusion_matrix, 'classification_report':class_report}

## Saving the best performing model

In [ ]:
save_path = Path('data') / 'output' / 'best_model_gpu.pickle'
save_path.parent.mkdir(parents=True, exist_ok=True)
dump(best_model, str(save_path))
dump(score, str(save_path.parent / f'best_model_gpu_score{save_path.suffix}'))